In [26]:
with open (r"C:\Users\Arun\Downloads\input.txt","r",encoding="utf-8") as f:
    text=f.read().lower()


print(len(text))
chars=sorted(set(text))
char2idx = { ch: idx for idx , ch in enumerate(chars) }
idx2char = { idx: ch for ch , idx in char2idx.items() }
print(idx2char)
print(char2idx)
print(len(text))
print(len(chars))


1115394
{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'a', 14: 'b', 15: 'c', 16: 'd', 17: 'e', 18: 'f', 19: 'g', 20: 'h', 21: 'i', 22: 'j', 23: 'k', 24: 'l', 25: 'm', 26: 'n', 27: 'o', 28: 'p', 29: 'q', 30: 'r', 31: 's', 32: 't', 33: 'u', 34: 'v', 35: 'w', 36: 'x', 37: 'y', 38: 'z'}
{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'a': 13, 'b': 14, 'c': 15, 'd': 16, 'e': 17, 'f': 18, 'g': 19, 'h': 20, 'i': 21, 'j': 22, 'k': 23, 'l': 24, 'm': 25, 'n': 26, 'o': 27, 'p': 28, 'q': 29, 'r': 30, 's': 31, 't': 32, 'u': 33, 'v': 34, 'w': 35, 'x': 36, 'y': 37, 'z': 38}
1115394
39


In [ ]:
seq_length = 100
step =1
sequences=[]
next_char=[]
for i in range(0,len(text)-seq_length,step):
    sequences.append([char2idx[c] for c in text[i:i+seq_length]])
    next_char.append(char2idx[text[i+seq_length]])
import numpy as np

from tensorflow.keras.utils import to_categorical
x = np.array(sequences)
y = to_categorical(next_char,num_classes=len(chars))
print(x.shape)

(29900, 100)


In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding

model = Sequential([
    Embedding(input_dim=len(chars),output_dim=64),
    LSTM(128),
    Dense(len(chars),activation="softmax")
])
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
model.build(input_shape=(None,seq_length))
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 100, 64)        │         2,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 36)             │         4,644 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 105,764 (413.14 KB)

 Trainable params: 105,764 (413.14 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
model.fit(x,y,batch_size=128,epochs=10)

Epoch 1/10
234/234 ━━━━━━━━━━━━━━━━━━━━ 16s 64ms/step - accuracy: 0.1639 - loss: 3.0744
Epoch 2/10
234/234 ━━━━━━━━━━━━━━━━━━━━ 14s 61ms/step - accuracy: 0.2946 - loss: 2.4433
Epoch 3/10
234/234 ━━━━━━━━━━━━━━━━━━━━ 14s 61ms/step - accuracy: 0.3322 - loss: 2.2839
Epoch 4/10
234/234 ━━━━━━━━━━━━━━━━━━━━ 14s 60ms/step - accuracy: 0.3684 - loss: 2.1699
Epoch 5/10
234/234 ━━━━━━━━━━━━━━━━━━━━ 14s 60ms/step - accuracy: 0.3879 - loss: 2.0900
Epoch 6/10
234/234 ━━━━━━━━━━━━━━━━━━━━ 14s 62ms/step - accuracy: 0.4041 - loss: 2.0310
Epoch 7/10
234/234 ━━━━━━━━━━━━━━━━━━━━ 15s 63ms/step - accuracy: 0.4241 - loss: 1.9611
Epoch 8/10
234/234 ━━━━━━━━━━━━━━━━━━━━ 15s 64ms/step - accuracy: 0.4374 - loss: 1.9144
Epoch 9/10
234/234 ━━━━━━━━━━━━━━━━━━━━ 14s 61ms/step - accuracy: 0.4506 - loss: 1.8589
Epoch 10/10
234/234 ━━━━━━━━━━━━━━━━━━━━ 15s 63ms/step - accuracy: 0.4559 - loss: 1.8166


In [2]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
def generate(seed_text, length=300, temperature=1.0):
    seed_text = seed_text.lower()
    generated = seed_text

    for _ in range(length):
        input_seq = [char2idx.get(c, 0) for c in generated[-seq_length:]]
        input_seq = pad_sequences([input_seq], maxlen=seq_length, padding='pre')

        preds = model.predict(input_seq, verbose=0)[0]

        preds = np.asarray(preds).astype('float64')
        preds = np.log(preds + 1e-8) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)

        next_index = np.random.choice(len(preds), p=preds)
        next_char = idx2char[next_index]

        
        generated += next_char

    return generated


In [3]:
print(generate("first citizen",length=99,temperature=0.1))

NameError: name 'seq_length' is not defined

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(generate))